In [ ]:
import numpy as np
import pandas as pd
import time
from collections import defaultdict 
from tensorflow import keras
from keras import layers

In [ ]:
!pip install tqdm

In [ ]:
import gensim, re
import numpy as np
import pandas as pd
import pickle
from os import listdir

from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import sys
import os

from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Embedding

In [ ]:
with open("/Data/viettel_train_input_no_tokenize.txt","r",encoding='UTF-8') as f:
  X_train=f.read().splitlines()
with open("/Data/viettel_test_input_no_tokenize.txt","r",encoding='UTF-8') as f:
  X_test=f.read().splitlines()
with open("/Data/viettel_train_label.txt","r",encoding='UTF-8') as f:
  y_train=f.read().splitlines()
with open("/Data/viettel_test_label.txt","r",encoding='UTF-8') as f:
  y_test=f.read().splitlines()

In [ ]:
def txtTokenizer(texts):
    tokenizer = Tokenizer()
    # fit the tokenizer on our text
    tokenizer.fit_on_texts(texts)

    # get all words that the tokenizer knows
    word_index = tokenizer.word_index
    return tokenizer, word_index

In [ ]:
tokenizer, word_index = txtTokenizer(X_train)

In [ ]:
X_train_encoded = tokenizer.texts_to_sequences(X_train)
X_train_encoded= pad_sequences(X_train_encoded)
X_test_encoded = tokenizer.texts_to_sequences(X_test)
X_test_encoded= pad_sequences(X_test_encoded,maxlen=X_train_encoded.shape[1])

In [ ]:
labels=list(set(y_train))
label2id=dict([label,id] for id,label in enumerate(labels))
y_train_vectorized=np.array([label2id[label] for label in y_train])
y_test_vectorized=np.array([label2id[label] for label in y_test])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
vocab_size=len(word_index)
embedding_size=300
model2 = Sequential()
model2.add(Embedding(vocab_size,embedding_size,input_length=X_train_encoded.shape[1]))
model2.add(LSTM(128,return_sequences=True))
model2.add(LSTM(256,return_sequences=False))
model2.add(Dense(len(labels),activation="softmax"))
model2.summary()
model2.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=['acc'])
checkpoint_filepath="/LSTM-weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
batch = 64
epochs = 40
checkpoint=ModelCheckpoint(checkpoint_filepath, 
                monitor = 'val_acc', 
                verbose = 1, 
                save_best_only = True, 
                mode = 'max')
callbacks_list = [checkpoint]
history2=model2.fit(X_train_encoded,y_train_vectorized,batch,epochs,validation_data=(X_test_encoded,y_test_vectorized),callbacks = callbacks_list)

In [ ]:
model2.load_weights("/LSTM-weights-improvement-35-0.84.hdf5")
start_time=time.time()
pred = model2.predict(X_test_encoded) 
print("Inference in {} seconds".format((time.time()-start_time)/len(X_test_encoded)))


In [ ]:
from matplotlib import pyplot as plt
plt.plot(history2.history['acc'])
plt.plot(history2.history['val_acc'])
plt.title('accuracy')
plt.xlabel('epoch')
plt.legend(['acc', 'val_acc'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
cnf_matrix = confusion_matrix(y_test_vectorized, pred)
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
df_cm = pd.DataFrame(cnf_matrix)
plt.figure(figsize=(30,10))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size

plt.show()